In [1]:
import numpy as np
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import preprocessing

sns.set_style('whitegrid')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
barcelona = pd.read_csv('/Users/jyots/Desktop/DSI-SF-4-Jyojha/Capstone_Project/airbnb_DataSets/Airbnb listings in Barcelona (November 2015).csv',
                   encoding='utf8')

In [3]:
barcelona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14012 entries, 0 to 14011
Data columns (total 16 columns):
Unnamed: 0              14012 non-null int64
room_id                 14012 non-null int64
host_id                 14012 non-null int64
room_type               14004 non-null object
borough                 0 non-null float64
neighborhood            14012 non-null object
reviews                 14012 non-null int64
overall_satisfaction    11311 non-null float64
accommodates            11330 non-null float64
bedrooms                13981 non-null float64
bathrooms               12875 non-null float64
price                   14012 non-null float64
minstay                 12961 non-null float64
latitude                14012 non-null float64
longitude               14012 non-null float64
collected               14012 non-null object
dtypes: float64(9), int64(4), object(3)
memory usage: 1.7+ MB


In [5]:
barcelona.drop('borough',axis=1,inplace=True)

In [6]:
barcelona.neighborhood.unique()

array([u'Ciutat Vella', u'Eixample', u'Sant Mart\ufffd', u'Nou Barris',
       u'Horta-Guinard\ufffd', u'Gr\ufffdcia', u'Sants-Montju\ufffdc',
       u'Sant Andreu', u'Sarri\ufffd-Sant Gervasi', u'Les Corts'], dtype=object)